In [ ]:
import hr
from bokeh.plotting import show
hr.setup_notebook()

In [ ]:
import numpy as np

from bokeh.plotting import figure, show, output_file

N = 500
x = np.linspace(0, 10, N)
y = np.linspace(0, 10, N)
xx, yy = np.meshgrid(x, y)
d = np.sin(xx)*np.cos(yy)

p = figure(x_range=(0, 10), y_range=(0, 10))

# must give a vector of image data for image parameter
p.image(image=[d], x=0, y=0, dw=10, dh=10, palette="Spectral11")

show(p)  # open a browser

In [ ]:
import numpy as np

from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.sampledata.stocks import AAPL, GOOG, IBM, MSFT

def datetime(x):
    return np.array(x, dtype=np.datetime64)

p1 = figure(x_axis_type="datetime", title="Stock Closing Prices")
p1.grid.grid_line_alpha=0.3
p1.xaxis.axis_label = 'Date'
p1.yaxis.axis_label = 'Price'

p1.line(datetime(AAPL['date']), AAPL['adj_close'], color='#A6CEE3', legend='AAPL')
p1.line(datetime(GOOG['date']), GOOG['adj_close'], color='#B2DF8A', legend='GOOG')
p1.line(datetime(IBM['date']), IBM['adj_close'], color='#33A02C', legend='IBM')
p1.line(datetime(MSFT['date']), MSFT['adj_close'], color='#FB9A99', legend='MSFT')
p1.legend.location = "top_left"

aapl = np.array(AAPL['adj_close'])
aapl_dates = np.array(AAPL['date'], dtype=np.datetime64)

window_size = 30
window = np.ones(window_size)/float(window_size)
aapl_avg = np.convolve(aapl, window, 'same')

p2 = figure(x_axis_type="datetime", title="AAPL One-Month Average")
p2.grid.grid_line_alpha = 0
p2.xaxis.axis_label = 'Date'
p2.yaxis.axis_label = 'Price'
p2.ygrid.band_fill_color = "olive"
p2.ygrid.band_fill_alpha = 0.1

p2.circle(aapl_dates, aapl, size=4, legend='close',
          color='darkgrey', alpha=0.2)

p2.line(aapl_dates, aapl_avg, legend='avg', color='navy')
p2.legend.location = "top_left"

show(gridplot([[p1,p2]], plot_width=400, plot_height=400))  # open a browser

In [ ]:
import os
from six.moves import urllib
import zipfile
from bokeh.util.string import encode_utf8


def extract_hosted_zip(data_url, save_dir, exclude_term=None):
    """Downloads, then extracts a zip file."""

    zip_name = os.path.join(save_dir, 'temp.zip')

    # get the zip file
    try:
        print('Downloading %r to %r' % (data_url, zip_name))
        zip_name, hdrs = urllib.request.urlretrieve(url=data_url, filename=zip_name)
        print('Download successfully completed')
    except IOError as e:
        print("Could not successfully retrieve %r" % data_url)
        raise e

    # extract, then remove temp file
    extract_zip(zip_name=zip_name, exclude_term=exclude_term)
    os.unlink(zip_name)
    print("Extraction Complete")


def extract_zip(zip_name, exclude_term=None):
    """Extracts a zip file to its containing directory."""

    zip_dir = os.path.dirname(os.path.abspath(zip_name))

    try:
        with zipfile.ZipFile(zip_name) as z:

            # write each zipped file out if it isn't a directory
            files = [zip_file for zip_file in z.namelist() if not zip_file.endswith('/')]

            print('Extracting %i files from %r.' % (len(files), zip_name))
            for zip_file in files:

                # remove any provided extra directory term from zip file
                if exclude_term:
                    dest_file = zip_file.replace(exclude_term, '')
                else:
                    dest_file = zip_file

                dest_file = os.path.normpath(os.path.join(zip_dir, dest_file))
                dest_dir = os.path.dirname(dest_file)

                # make directory if it does not exist
                if not os.path.isdir(dest_dir):
                    os.makedirs(dest_dir)

                # read file from zip, then write to new directory
                data = z.read(zip_file)
                with open(dest_file, 'wb') as f:
                    f.write(encode_utf8(data))

    except zipfile.error as e:
        print("Bad zipfile (%r): %s" % (zip_name, e))
        raise e


# info for retrieving and extracting the zip file
this_dir = os.path.join(os.path.dirname(os.path.realpath('.')), 'notebooks', 'data')
zip_file = 'http://quantquote.com/files/quantquote_daily_sp500_83986.zip'
zip_dir = 'quantquote_daily_sp500_83986/'
print(this_dir)
extract_hosted_zip(data_url=zip_file, save_dir=this_dir, exclude_term=zip_dir)

In [ ]:
def stocks(doc):
  ''' Create a simple stocks correlation dashboard.
  Choose stocks to compare in the drop down widgets, and make selections
  on the plots to update the summary and histograms accordingly.
  .. note::
      Running this example requires downloading sample data. See
      the included `README`_ for more information.
  Use the ``bokeh serve`` command to run the example by executing:
      bokeh serve stocks
  at your command prompt. Then navigate to the URL
      http://localhost:5006/stocks
  .. _README: https://github.com/bokeh/bokeh/blob/master/examples/app/stocks/README.md
  '''
  try:
      from functools import lru_cache
  except ImportError:
      # Python 2 does stdlib does not have lru_cache so let's just
      # create a dummy decorator to avoid crashing
      print ("WARNING: Cache for this example is available on Python 3 only.")
      def lru_cache():
        def dec(f):
          def _(*args, **kws):
            return f(*args, **kws)
          return _
        return dec

  from os.path import dirname, join, realpath

  import pandas as pd

  from bokeh.layouts import row, column
  from bokeh.models import ColumnDataSource
  from bokeh.models.widgets import PreText, Select
  from bokeh.plotting import figure

  DATA_DIR = join(dirname(realpath('.')), 'notebooks', 'data', 'daily')

  DEFAULT_TICKERS = ['AAPL', 'GOOG', 'INTC', 'BRCM', 'YHOO']

  def nix(val, lst):
      return [x for x in lst if x != val]

  @lru_cache()
  def load_ticker(ticker):
      fname = join(DATA_DIR, 'table_%s.csv' % ticker.lower())
      data = pd.read_csv(fname, header=None, parse_dates=['date'],
                         names=['date', 'foo', 'o', 'h', 'l', 'c', 'v'])
      data = data.set_index('date')
      return pd.DataFrame({ticker: data.c, ticker+'_returns': data.c.diff()})

  @lru_cache()
  def get_data(t1, t2):
      df1 = load_ticker(t1)
      df2 = load_ticker(t2)
      data = pd.concat([df1, df2], axis=1)
      data = data.dropna()
      data['t1'] = data[t1]
      data['t2'] = data[t2]
      data['t1_returns'] = data[t1+'_returns']
      data['t2_returns'] = data[t2+'_returns']
      return data

  # set up widgets

  stats = PreText(text='', width=500)
  ticker1 = Select(value='AAPL', options=nix('GOOG', DEFAULT_TICKERS))
  ticker2 = Select(value='GOOG', options=nix('AAPL', DEFAULT_TICKERS))

  # set up plots

  source = ColumnDataSource(data=dict(date=[], t1=[], t2=[], t1_returns=[], t2_returns=[]))
  source_static = ColumnDataSource(data=dict(date=[], t1=[], t2=[], t1_returns=[], t2_returns=[]))
  tools = 'pan,wheel_zoom,xbox_select,reset'

  corr = figure(plot_width=350, plot_height=350,
                tools='pan,wheel_zoom,box_select,reset')
  corr.circle('t1_returns', 't2_returns', size=2, source=source,
              selection_color="orange", alpha=0.6, nonselection_alpha=0.1, selection_alpha=0.4)

  ts1 = figure(plot_width=900, plot_height=200, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
  ts1.line('date', 't1', source=source_static)
  ts1.circle('date', 't1', size=1, source=source, color=None, selection_color="orange")

  ts2 = figure(plot_width=900, plot_height=200, tools=tools, x_axis_type='datetime', active_drag="xbox_select")
  ts2.x_range = ts1.x_range
  ts2.line('date', 't2', source=source_static)
  ts2.circle('date', 't2', size=1, source=source, color=None, selection_color="orange")

  # set up callbacks

  def ticker1_change(attrname, old, new):
      ticker2.options = nix(new, DEFAULT_TICKERS)
      update()

  def ticker2_change(attrname, old, new):
      ticker1.options = nix(new, DEFAULT_TICKERS)
      update()

  def update(selected=None):
      t1, t2 = ticker1.value, ticker2.value

      data = get_data(t1, t2)
      source.data = source.from_df(data[['t1', 't2', 't1_returns', 't2_returns']])
      source_static.data = source.data

      update_stats(data, t1, t2)

      corr.title.text = '%s returns vs. %s returns' % (t1, t2)
      ts1.title.text, ts2.title.text = t1, t2

  def update_stats(data, t1, t2):
      stats.text = str(data[[t1, t2, t1+'_returns', t2+'_returns']].describe())

  ticker1.on_change('value', ticker1_change)
  ticker2.on_change('value', ticker2_change)

  def selection_change(attrname, old, new):
      t1, t2 = ticker1.value, ticker2.value
      data = get_data(t1, t2)
      selected = source.selected.indices
      if selected:
        data = data.iloc[selected, :]
      update_stats(data, t1, t2)

  source.on_change('selected', selection_change)

  # set up layout
  widgets = column(ticker1, ticker2, stats)
  main_row = row(corr, widgets)
  series = column(ts1, ts2)
  layout = column(main_row, series)

  # initialize
  update()

  doc.add_root(layout)
  doc.title = "Stocks"
show(stocks)

In [ ]:
def crossfilter(doc):
  ''' https://github.com/bokeh/bokeh/tree/master/examples/app/crossfilter
      Modified for JupyterLab
  '''
  import pandas as pd

  from bokeh.layouts import row, widgetbox
  from bokeh.models import Select
  from bokeh.palettes import Spectral5
  from bokeh.plotting import curdoc, figure
  from bokeh.sampledata.autompg import autompg_clean as df

  df = df.copy()

  SIZES = list(range(6, 22, 3))
  COLORS = Spectral5

  # data cleanup
  df.cyl = df.cyl.astype(str)
  df.yr = df.yr.astype(str)
  del df['name']

  columns = sorted(df.columns)
  discrete = [x for x in columns if df[x].dtype == object]
  continuous = [x for x in columns if x not in discrete]
  quantileable = [x for x in continuous if len(df[x].unique()) > 20]

  def create_figure():
      xs = df[x.value].values
      ys = df[y.value].values
      x_title = x.value.title()
      y_title = y.value.title()

      kw = dict()
      if x.value in discrete:
        kw['x_range'] = sorted(set(xs))
      if y.value in discrete:
        kw['y_range'] = sorted(set(ys))
      kw['title'] = "%s vs %s" % (x_title, y_title)

      p = figure(plot_height=600, plot_width=800, tools='pan,box_zoom,reset', **kw)
      p.xaxis.axis_label = x_title
      p.yaxis.axis_label = y_title

      if x.value in discrete:
        p.xaxis.major_label_orientation = pd.np.pi / 4

      sz = 9
      if size.value != 'None':
        groups = pd.qcut(df[size.value].values, len(SIZES))
        sz = [SIZES[xx] for xx in groups.codes]

      c = "#31AADE"
      if color.value != 'None':
        groups = pd.qcut(df[color.value].values, len(COLORS))
        c = [COLORS[xx] for xx in groups.codes]
      p.circle(x=xs, y=ys, color=c, size=sz, line_color="white", alpha=0.6, hover_color='white', hover_alpha=0.5)

      return p


  def update(attr, old, new):
      layout.children[1] = create_figure()


  x = Select(title='X-Axis', value='mpg', options=columns)
  x.on_change('value', update)

  y = Select(title='Y-Axis', value='hp', options=columns)
  y.on_change('value', update)

  size = Select(title='Size', value='None', options=['None'] + quantileable)
  size.on_change('value', update)

  color = Select(title='Color', value='None', options=['None'] + quantileable)
  color.on_change('value', update)

  controls = widgetbox([x, y, color, size], width=200)
  layout = row(controls, create_figure())

  doc.add_root(layout)
  doc.title = "Crossfilter"
show(crossfilter)

In [ ]:
def weather(doc):
  ''' https://github.com/bokeh/bokeh/tree/master/examples/app/weather
      Modified for JupyterLab
  '''
  from os.path import join, dirname
  import datetime

  import pandas as pd
  from scipy.signal import savgol_filter

  from bokeh.io import curdoc
  from bokeh.layouts import row, column
  from bokeh.models import ColumnDataSource, DataRange1d, Select
  from bokeh.palettes import Blues4
  from bokeh.plotting import figure

  STATISTICS = ['record_min_temp', 'actual_min_temp', 'average_min_temp', 'average_max_temp', 'actual_max_temp', 'record_max_temp']

  def get_dataset(src, name, distribution):
      df = src[src.airport == name].copy()
      del df['airport']
      df['date'] = pd.to_datetime(df.date)
      # timedelta here instead of pd.DateOffset to avoid pandas bug < 0.18 (Pandas issue #11925)
      df['left'] = df.date - datetime.timedelta(days=0.5)
      df['right'] = df.date + datetime.timedelta(days=0.5)
      df = df.set_index(['date'])
      df.sort_index(inplace=True)
      if distribution == 'Smoothed':
        window, order = 51, 3
        for key in STATISTICS:
          df[key] = savgol_filter(df[key], window, order)

      return ColumnDataSource(data=df)

  def make_plot(source, title):
      plot = figure(x_axis_type="datetime", plot_width=800, tools="", toolbar_location=None)
      plot.title.text = title

      plot.quad(top='record_max_temp', bottom='record_min_temp', left='left', right='right',
                color=Blues4[2], source=source, legend="Record")
      plot.quad(top='average_max_temp', bottom='average_min_temp', left='left', right='right',
                color=Blues4[1], source=source, legend="Average")
      plot.quad(top='actual_max_temp', bottom='actual_min_temp', left='left', right='right',
                color=Blues4[0], alpha=0.5, line_color="black", source=source, legend="Actual")

      # fixed attributes
      plot.xaxis.axis_label = None
      plot.yaxis.axis_label = "Temperature (F)"
      plot.axis.axis_label_text_font_style = "bold"
      plot.x_range = DataRange1d(range_padding=0.0)
      plot.grid.grid_line_alpha = 0.3

      return plot

  def update_plot(attrname, old, new):
      city = city_select.value
      plot.title.text = "Weather data for " + cities[city]['title']

      src = get_dataset(df, cities[city]['airport'], distribution_select.value)
      source.data.update(src.data)

  city = 'Austin'
  distribution = 'Discrete'

  cities = {
      'Austin': {
        'airport': 'AUS',
        'title': 'Austin, TX',
      },
      'Boston': {
        'airport': 'BOS',
        'title': 'Boston, MA',
      },
      'Seattle': {
        'airport': 'SEA',
        'title': 'Seattle, WA',
      }
  }

  city_select = Select(value=city, title='City', options=sorted(cities.keys()))
  distribution_select = Select(value=distribution, title='Distribution', options=['Discrete', 'Smoothed'])

  def extract_hosted_data(data_url, file_name):
    """Downloads data file."""
    try:
        print('Downloading %r to %r' % (data_url, file_name))
        file_name, hdrs = urllib.request.urlretrieve(url=data_url, filename=file_name)
        print('Download successfully completed')
    except IOError as e:
        print("Could not successfully retrieve %r" % data_url)
        raise e

  data_location = os.path.join('.', 'data/2015_weather.csv')
  extract_hosted_data('https://raw.githubusercontent.com/bokeh/bokeh/master/examples/app/weather/data/2015_weather.csv',
                      data_location)
  df = pd.read_csv(data_location)
  source = get_dataset(df, cities[city]['airport'], distribution)
  plot = make_plot(source, "Weather data for " + cities[city]['title'])

  city_select.on_change('value', update_plot)
  distribution_select.on_change('value', update_plot)

  controls = column(city_select, distribution_select)

  doc.add_root(row(plot, controls))
  doc.title = "Weather"
show(weather)

In [ ]:
# WebGL examples showcasing performance with N >= 10000.

In [ ]:
import numpy as np

from bokeh.plotting import figure, show

N = 10000

x = np.random.normal(0, np.pi, N)
y = np.sin(x) + np.random.normal(0, 0.2, N)

TOOLS = "pan,wheel_zoom,box_zoom,reset,save,box_select"

p = figure(tools=TOOLS, output_backend="webgl")

p.circle(x, y, alpha=0.1, nonselection_alpha=0.001)

show(p)

In [ ]:
""" A line with 10k points to show off the WebGL line implementation.
"""

import numpy as np

from bokeh.plotting import figure, show, output_file

N = 10000
x = np.linspace(0, 10*np.pi, N)
y = np.cos(x) + np.sin(2*x+1.25) + np.random.normal(0, 0.001, (N, ))

p = figure(title="A line consisting of 10k points", output_backend="webgl")

p.line(x, y, color="#22aa22", line_width=3)

show(p)